<small>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones, FaMAF UNC, 2021.</small>
# Exploración y Curación de Datos 

Este trabajo pertenece a la mentoría **GAP: Gestión Automática de Pedidos de Combustibles Líquidos basada en telemetría de inventarios**.

Integrantes:
- Bernaschini, María Laura
- Bosch, Daniela

## Introducción

En el trabajo anterior se hicieron algunos análisis de los datos teniendo en cuenta el dataset completo y las agrupaciones por tanque. En este trabajo, buscamos explorar un poco más los datos haciendo curación y análisis más complejos.

El filtrado de datos erróneos se hará por tanque, mientras que el análisis y procesamiento se hará agrupando por centro operativo y producto.

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn
from datetime import datetime, timedelta
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.impute import SimpleImputer
from statsmodels.tsa.seasonal import seasonal_decompose

seaborn.set_context('talk')
# Set float format
pd.set_option('display.float_format','{:.2f}'.format)

# Set style
seaborn.set_style("darkgrid")
seaborn.set_palette('pastel')
seaborn.set_context("paper", rc={"font.size":12,"axes.titlesize":12,"axes.labelsize":12}) 

ModuleNotFoundError: No module named 'pydrive'

## Lectura del dataset

Teniendo en cuenta el análisis que se hizo previamente procedemos a levantar el dataset con los tipos correctos y sólamente las columnas que vamos a utilizar.

In [ ]:
dtypes = {
    "id_equipo": "category",
    "id_tanque": "category",
    "nombre_producto": "category",
    "industria": "category",
}
parse_dates = ["timestamp"]
usecols = [
    'timestamp', 'id_equipo', 'id_tanque', 'nombre_producto', 'volumen',
    'fuel_level_dmm', 'vbat2', 'capacidad', 'industria', 'codigo'
]

In [ ]:
def connect_to_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    return GoogleDrive(gauth)

def read_pkl_from_drive(drive, id, filename):
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile(filename)  
    return pd.read_pickle(filename)

def read_csv_from_drive(drive, id, filename, usecols, parse_dates, dtypes):
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile(filename)  
    return pd.read_csv(
        filename, 
        dtype=dtypes, 
        parse_dates=parse_dates,
        usecols=usecols
    )

drive = connect_to_drive()

In [ ]:
filename = '14KRbp2KPwMK-k_6pjD5FmbpVZ0AudBZR'
raw_df = read_csv_from_drive(
    drive, 
    filename, 
    'StorageInventory_2021_Q1.csv',
    usecols,
    parse_dates,
    dtypes
)
raw_df['volumen'] = raw_df['volumen'].astype(numpy.float32)
raw_df['capacidad'] = raw_df['capacidad'].astype(numpy.float32)
raw_df.head()

In [ ]:
# Mantenemos el original sin modificaciones
df = raw_df.copy()

## Preprocesamiento y análisis de los datos

### Agrupación por tanques

Como se vio en el trabajo anterior, fue necesario agrupar los historiales por sensor (por tanque) para realizar algunos filtrados y algunos análisis.

Para filtrados referidos a errores en los sensores, se utilizará esta agrupación, para el resto de los análisis, se agruparán por centro operativo y producto.

In [ ]:
# Ejemplo del uso de groupby
df.groupby('id_tanque').first()

In [ ]:
# Centro operativo + producto para facilitar las agrupaciones posteriores
df["equipo-producto"] = df["id_equipo"].astype(str) + "-" + df["nombre_producto"].astype(str)
df["equipo-producto"] = df["equipo-producto"].astype("category")
df.head()

En la gestión automática de pedidos es preferible no detectar la falta de combustible aunque el centro operativo esté en falta (error de tipo I) que predecir erróneamente la falta del mismo (error de tipo II), ya que la misma implicaría toda una logística con elevados costos. 

Se filtrarán registros o grupo de registros completos de productos y centros operativos según los criterios:
- Eliminación únicamente de registros erróneos si estos no superan los 50 casos.
- Eliminación de grupos producto-equipo completos si alguno de los tanques del grupo no reportó en un periodo mayor a 24hs **en las últimas dos semanas**. 
- Eliminación de grupos producto-equipo completos si alguno de los tanques del grupo presentan más de 50 casos de error. Si bien podría utilizarse el mismo criterio de únicamente ver las últimas dos semanas, consideramos que se trata de una **falla registrada por el equipo o la sonda**, por lo tanto, decidimos quitarlos si aparecen en algún momento de la serie de tiempo.

**No se realizarán predicciones** sobre tales grupos debido a que contienen datos erróneos o faltantes en uno o varios tanques.

In [ ]:
# Acumulamos los centros productos que eliminamos en este dataset
removed = pd.DataFrame({}, columns=["equipo-producto"])

### Equipos y productos dónde existen volúmenes de tanque menores a 0

In [ ]:
df[df.volumen < 0]

En este caso hay un sólo dato erróneo en el equipo `685`, por lo que sólo quitaremos el caso.

### Equipos con capacidad nan, negativo o cero

In [ ]:
cap_cero = df[df.capacidad <= 0]
cap_cero

In [ ]:
cap_nan = df[df.capacidad.isna()]
cap_nan["equipo-producto"] = cap_nan["equipo-producto"].cat.remove_unused_categories()
cap_nan

In [ ]:
r = pd.DataFrame()
r["equipo-producto"] = cap_nan["equipo-producto"].unique()
r["capacidad_nan"] = True

removed = pd.merge(removed, r, on="equipo-producto", how='outer')

### Equipos y productos dónde existen volúmenes que exceden la capacidad del tanque

In [ ]:
# TODO: ver por tanque, debería ser algo tipo df.groupby("id_tanque")["volumen"].max()
df[df["volumen"] > df["capacidad"].max()]

Los equipos que se obtuvieron fueron el `848` y el `859`. Al igual que en el caso anterior, debido a que la catidad de datos erróneos es escaso por equipo, sólo sacaremos los casos.

### Equipos y productos dónde la batería de los sensores es 0

In [ ]:
df_bat = df[df["vbat2"] == 0]
df_bat["equipo-producto"] = df_bat["equipo-producto"].cat.remove_unused_categories()
df_bat

Observamos algunos casos.

In [ ]:
bat = pd.DataFrame(df_bat["equipo-producto"].value_counts())
bat

Quiraremos los grupos: 
- `111` productos "BIO" y "Producto 0", 
- `106` "Producto 2", 
- `69` y 
- `848` producto "Diesel". 

En el caso de los equipos `353` y `946` sólo eliminaremos los casos ya que son pocos.

In [ ]:
r = pd.DataFrame()
r["equipo-producto"] = bat.index[:5].unique()
r["pilas_cero"] = True

removed = pd.merge(removed, r, on="equipo-producto", how='outer')

### Equipos y productos donde el volumen de combustible es 0 y el nivel mayor a 0

In [ ]:
df_fuel = df[(df.volumen == 0) & (df.fuel_level_dmm > 0)]
df_fuel["equipo-producto"] = df_fuel["equipo-producto"].cat.remove_unused_categories()
df_fuel

Observamos algunos casos.

In [ ]:
fuel = pd.DataFrame(df_fuel["equipo-producto"].value_counts())
fuel

En esta oportunidad, quitaremos los equipos `685`, `534`, `843`, `356` y `763`, y en el caso de los equipos `848`, `577` y `905` eliminaremos los casos ya que son pocos.

In [ ]:
r = pd.DataFrame()
r["equipo-producto"] = fuel.index[:5].unique()
r["volumen_cero_fuel_nocero"] = True

removed = pd.merge(removed, r, on="equipo-producto", how='outer')

### Equipos y productos donde el eco de las sondas es 0

In [ ]:
# Separamos el código en dos datos
df['c'] = df['codigo'].apply(lambda x: str(x)[0])
df['echoes'] = pd.to_numeric(df['codigo'].apply(lambda x: str(x)[1]), errors='coerce')

In [ ]:
df_echoes = df[df["echoes"] == 0]
df_echoes["equipo-producto"] = df_echoes["equipo-producto"].cat.remove_unused_categories()
df_echoes[:10]

In [ ]:
pd.DataFrame(df_echoes["equipo-producto"].value_counts())[:20]

Graficamos algunos ejemplos.

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, sharey=False, figsize=(20, 7))
ej1 = df.query("`equipo-producto` == '646-Gas Oil'")
ej2 = df.query("`equipo-producto` == '175-Product 1'")
ej1.loc[:,'id_tanque'] = ej1['id_tanque'].cat.remove_unused_categories()
ej2.loc[:,'id_tanque'] = ej2['id_tanque'].cat.remove_unused_categories()

seaborn.lineplot(
    data=ej1, 
    y='volumen', 
    x='timestamp', 
    hue='id_tanque',
    ax=axes[0]
).axes.set(ylim=(0, None))
seaborn.lineplot(
    data=ej2, 
    y='volumen', 
    x='timestamp', 
    hue='id_tanque',
    ax=axes[1]
).axes.set(ylim=(0, None))
seaborn.despine()

Para este caso, definimos un criterio de eliminación ya que son muchos los equipos con ecos 0. Como se muestra más adelante, se quitaron aquellos equipos que presentaban más de 50 ecos 0.  

In [ ]:
ecos = pd.DataFrame(df[df["echoes"] == 0]["equipo-producto"])
ecos.loc[:,"equipo-producto"] = ecos["equipo-producto"].cat.remove_unused_categories()
ecos = ecos.groupby("equipo-producto").first()[ecos["equipo-producto"].value_counts() > 50]
ecos

In [ ]:
r = pd.DataFrame()
r["equipo-producto"] = ecos.index
r["ecos_cero"] = True

removed = pd.merge(removed, r, on="equipo-producto", how='outer')

### Equipos y productos donde el eco de las sondas es mayor a 2

In [ ]:
df[df["echoes"] > 2]

### Equipos con códigos de error

In [ ]:
error_codes = ['m', 'M', 'F']
df.query("c in @error_codes")

Para este caso, definimos un criterio de eliminación ya que son muchos los equipos con códigos de error. Como se muestra más adelante, se quitaron aquellos equipos que presentaban más de 50 registros con códigos de error.  

In [ ]:
# Equipos con un conteo de valores de código de error mayor a 50
error = pd.DataFrame(df[df["c"].isin(error_codes)]["equipo-producto"])
error.loc[:,"equipo-producto"] = error["equipo-producto"].cat.remove_unused_categories()
error = error.groupby("equipo-producto").first()[error["equipo-producto"].value_counts() > 50]
error

In [ ]:
r = pd.DataFrame()
r["equipo-producto"] = error.index
r["codigo_error"] = True

removed = pd.merge(removed, r, on="equipo-producto", how='outer')

## Filtrado de casos y equipos

Filtramos todos los casos y los grupos mencionados anteriormente.

In [ ]:
len(df)

In [ ]:
# Casos
df = df[~(df.volumen < 0)]
df = df[~(df.volumen > df.capacidad.max())]
df = df[~(df.echoes > 2)]
df = df[~(df.echoes == 0)]
df = df[~(df["c"].isin(error_codes))]

# Grupos centro producto
df = df[~(df["equipo-producto"].isin(removed["equipo-producto"]))]

In [ ]:
len(df)

Probamos si el filtrado por equipo en relación a los ecos se realizó correctamente

In [ ]:
df[df.id_equipo=="646"]

In [ ]:
df[df.id_equipo=="619"]

## Filtrado de centros por los gaps
Debido a la eliminación de datos erróneos, se generaron espacios de tiempo sin datos en algunos tanques. Otro motivo por el cual puedan existir estos gaps, es debido a que la sonda o la consola no reportan a la nube, por problemas de conectividad, entre otros.


Se filtrarán centros operativos en donde al menos uno de los tanques del grupo tiene un espacio de tiempo sin datos de al menos 24hs.

In [ ]:
df['delta_timestamp'] = df.groupby('id_tanque')['timestamp'].diff()

In [ ]:
df.timestamp.max()

In [ ]:
cond3 = (df['delta_timestamp'] > '1 days')
cond4 = (df['timestamp'] > datetime(2021, 3, 15, 23, 59, 58))
cond5 = (df['timestamp'] < datetime(2021, 3, 31, 23, 59, 58))

gaps = pd.DataFrame(df[cond3 & cond4 & cond5]["equipo-producto"].unique())
gaps

Graficamos un ejemplo

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, sharey=False, figsize=(20, 7))
ej1 = df.query("`equipo-producto` == '895-Producto 0'")
ej2 = df.query("`equipo-producto` == '653-GAS OIL'")
ej1.loc[:,'id_tanque'] = ej1['id_tanque'].cat.remove_unused_categories()
ej2.loc[:,'id_tanque'] = ej2['id_tanque'].cat.remove_unused_categories()

seaborn.scatterplot(
    data=ej1, 
    y='volumen', 
    x='timestamp', 
    hue='id_tanque',
    ax=axes[0]
).axes.set(ylim=(0, None))
seaborn.lineplot(
    data=ej2, 
    y='volumen', 
    x='timestamp', 
    hue='id_tanque',
    ax=axes[1]
).axes.set(ylim=(0, None))
seaborn.despine()

In [ ]:
r = pd.DataFrame()
r["equipo-producto"] = gaps[0]
r["gaps"] = True

removed = pd.merge(removed, r, on="equipo-producto", how='outer')

In [ ]:
df = df[~(df["equipo-producto"].isin(removed["equipo-producto"]))]

Finalmente, los centros operativos - productos eliminados fueron:

In [ ]:
removed = removed.fillna(False)
removed

## Uniformidad de la serie temporal

Para poder hacer las agregaciones necesarias, necesitamos realizar un resampleo de los datos por una unidad de tiempo uniforme, de lo contrario, los datos no pueden ser comparables y agregables entre tanques.

Vemos además que antes de realizar el sampleo, será necesario filtrar los registros duplicados. Esto se debe al reporte de dos mediciones en el mismo timestamp con dos códigos distintos. Para esta etapa no los necesitamos y los erróneos fueron eliminados previamente, por lo tanto, podemos quitar uno de ellos.

In [ ]:
# Ejemplo de un tanque que reportó duplicado
df.query("id_tanque == '1010'").tail()

In [ ]:
resampled_df = df.groupby('id_tanque', as_index=False)\
    .apply(
        lambda x : x.drop_duplicates('timestamp')\
                .set_index('timestamp')\
                .resample('5min')\
                .bfill())\
    .reset_index()\
    .set_index('timestamp')\
    .drop(columns=['level_0'])
resampled_df.head()

Graficamos un ejemplo

In [ ]:
co_900 = resampled_df[resampled_df.id_equipo == "900"]
# Debemos remover las categorías no utilizadas
co_900.loc[:,'id_tanque'] = co_900['id_tanque']\
    .cat.remove_unused_categories()
co_900.loc[:,'nombre_producto'] = co_900['nombre_producto']\
    .cat.remove_unused_categories()

In [ ]:
n = co_900['id_tanque'].nunique()
fig, axes = plt.subplots(n, 1, sharex=True, sharey=True, figsize=(20, 10))

for i, tanque in enumerate(co_900['id_tanque'].unique()):
    seaborn.lineplot(
        data=co_900.query("id_tanque == @tanque"), 
        y='volumen', 
        x='timestamp',
        hue='nombre_producto', 
        ax=axes[i]
    )
seaborn.despine()

## Consumos por producto y centros operativos

Para poder calcular los consumos, debemos sumar todas las diferencias negativas (primera derivada) por centro operativo y producto. 

A veces puede haber ruido en el volumen histórico de los tanques que pueden confundirse con movimientos de combustible, por lo tanto, no sería correcto utilizar las diferencias entre volúmenes en el tiempo sin ningún tratamiento previo. Necesitamos suavizar los registros para obtener las diferencias reales. Para obtener el cálculo de consumos más preciso se utilizan las **direfencias entre las medias móviles**. 

In [ ]:
resampled_df['moving_avg'] = resampled_df.groupby('id_tanque')['volumen'].transform(
    lambda x: x.rolling(window=4).mean()
)

In [ ]:
resampled_df['volumen_diff'] = resampled_df.groupby('id_tanque')['moving_avg'].diff()

In [ ]:
resampled_df['consumo'] = numpy.where(
    resampled_df['volumen_diff'] < 0, 
    resampled_df['volumen_diff'].abs(), 
    0
)

In [ ]:
resampled_df.tail()

### Agrupación por centro operativo y producto

El objetivo del proyecto es predecir cuándo un centro operativo va a quedarse sin combustible, por lo tanto la predicción por tanque no sería correcta ya que no se justificaría generar toda una logística de pedido del producto por un solo tanque.

Una vez filtrados los datos, necesitamos sumar las capacidades y los volúmenes de los tanques por centro operativo y por producto.



In [ ]:
grouped_prod_equipo = resampled_df\
    .groupby(['id_equipo', 'nombre_producto', 'timestamp'], observed=True)\
    .agg({
        'capacidad':'sum', 
        'volumen':'sum', 
        "consumo": "sum",
        'industria': 'first',
    })\
    .reset_index()\
    .set_index('timestamp')
grouped_prod_equipo.head()


Graficamos el mismo centro operativo de ejemplo.

In [ ]:
co_900_grouped = grouped_prod_equipo.query("id_equipo == '900'")
# Debemos remover las categorías no utilizadas
co_900_grouped.loc[:,'nombre_producto'] = co_900_grouped['nombre_producto']\
    .cat.remove_unused_categories()

In [ ]:
n = co_900_grouped['nombre_producto'].nunique()
fig, axes = plt.subplots(n, 1, sharex=True, sharey=False, figsize=(20, 10))

for i, producto in enumerate(co_900_grouped['nombre_producto'].unique()):
    seaborn.lineplot(
        data=co_900_grouped.query("nombre_producto == @producto"), 
        y='volumen', 
        x='timestamp', 
        hue='nombre_producto', 
        ax=axes[i]
    ).axes.set(ylim=(0, None))
seaborn.despine()

## Imputación de datos faltantes

In [ ]:
grouped_prod_equipo.info()

In [ ]:
grouped_prod_equipo.isna().sum()

In [ ]:
group_ind_na=grouped_prod_equipo[grouped_prod_equipo.industria.isna()]
group_ind_na[:6]

In [ ]:
group_ind_na.id_equipo.unique()

In [ ]:
grouped_prod_equipo[grouped_prod_equipo.id_equipo=="554"]

Se imputarán con la constante "Unknown" los valores nulos de industria ya que, como se observó en el práctico anterior, no corresponden a una identificación de la industria, por lo cual, no hay posibilidad de imputar estos valores.

In [ ]:
# grouped_prod_equipo.dropna(subset=["industria"], axis=0, inplace=True)
imputer = SimpleImputer(missing_values=numpy.nan, strategy='constant', fill_value='Unknown')
X = imputer.fit_transform(grouped_prod_equipo)
grouped_prod_equipo = pd.DataFrame(X, columns=grouped_prod_equipo.columns, index=grouped_prod_equipo.index)
grouped_prod_equipo

In [ ]:
grouped_prod_equipo.isna().sum()

In [ ]:
grouped_prod_equipo.info()

## Escalado del volumen y consumo por la capacidad por centro-producto

In [ ]:
grouped_prod_equipo["volumen_escl"] = grouped_prod_equipo["volumen"]/grouped_prod_equipo["capacidad"]

In [ ]:
grouped_prod_equipo["consumo_escl"] = grouped_prod_equipo["consumo"]/grouped_prod_equipo["capacidad"]

In [ ]:
grouped_prod_equipo[:6]

## Eliminación de columnas

Una vez completa la curación, se eliminan las columnas que ya no se van a necesitar.

In [ ]:
grouped_prod_equipo.drop(columns=["volumen", "consumo"], axis=1, inplace=True)

In [ ]:
grouped_prod_equipo.head()

## Guardado de los modelos

Finalmente guardamos los datos procesados en formato pickle para ser utilizado en trabajos posteriores.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Colab Notebooks/DiploDatos/Mentoría/Datasets/curated_df.pkl'
# path = '/content/drive/MyDrive/Colab Notebooks/Diplo_Datos/Mentoria_GAP/curated_df.pkl'
grouped_prod_equipo.to_pickle(path)

Levantamos el archivo pickle y graficamos la última semana de alguno de los modelos.

In [ ]:
pkl_key = '11ihPPpzsS_9R5Kb--ZQNuDjFTzYkgc4C'
curated_pkl = read_pkl_from_drive(drive, pkl_key, 'curated_df.pkl')

In [ ]:
curated_pkl.head()

In [ ]:
curated_pkl.id_equipo.value_counts()

In [ ]:
curated_pkl[curated_pkl.id_equipo=="248"]

In [ ]:
co_248_d500 = curated_pkl[(curated_pkl["id_equipo"]=="248") & (curated_pkl["nombre_producto"]=="D500")] 
co_248_d500.head()

In [ ]:
co_248_d500.dtypes

In [ ]:
fig, axes = plt.subplots(1, 1, sharex=True, sharey=False, figsize=(20, 4))

seaborn.lineplot(
    x=co_248_d500.index, 
    y=co_248_d500["volumen_escl"].astype(float), 
).axes.set(ylim=(0, None))
seaborn.despine()

## Análisis

Luego del trabajo de curación y guardado de los datos, realizamos algunos análisis  postprocesamiento para ser tenidos en cuenta en los próximos trabajos.

### Estacionalidad de la serie

Para detectar cuándo un centro operativo se va a quedar sin combustible no es suficiente con extrapolar la serie con las últimas pendientes (consumos) ya que el comportamiento de cada centro depende del  día, turno, horario, etc. Necesitamos analizar la **estacionalidad** de la misma.

Utilizamos del paquete de `statsmodels` el método `seasonal_descompose` para descomponer la serie y analizar su estacionalidad. Si bien este método es Naïve, por el momento nos resulta suficiente para obtener algunos análisis (y quizás hiperparámetros) preliminares.

Ejemplo con un equipo de la industria Estación de Servicio (248) y el producto D500

In [ ]:
decomp_volumen=seasonal_decompose(co_248_d500["volumen_escl"],freq= 60*24)   

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=4, sharex=True, figsize=(16,8))

for i, column in enumerate(['volumen_escl', 'consumo_escl']):
    
    res = seasonal_decompose(co_248_d500[column], freq=60*24, model='additive', extrapolate_trend='freq')

    ax[0,i].set_title('Decomposition of {}'.format(column), fontsize=16)
    res.observed.plot(ax=ax[0,i], legend=False, color='dodgerblue')
    ax[0,i].set_ylabel('Observed', fontsize=14)

    res.trend.plot(ax=ax[1,i], legend=False, color='dodgerblue')
    ax[1,i].set_ylabel('Trend', fontsize=14)

    res.seasonal.plot(ax=ax[2,i], legend=False, color='dodgerblue')
    ax[2,i].set_ylabel('Seasonal', fontsize=14)
    
    res.resid.plot(ax=ax[3,i], legend=False, color='dodgerblue')
    ax[3,i].set_ylabel('Residual', fontsize=14)


Vamos a analizar el comportamiento respecto a:
- Momento de la semana.
- Horario laboral habitual.
- Turnos mañana, tarde y noche.

Observamos ejemplos sobre dos grandes grupos de producto (Diesel y Nafta).

In [ ]:
co_900_nafta = curated_pkl[(curated_pkl["id_equipo"] == "900") & (curated_pkl["nombre_producto"] == "Nafta Super")] 
co_900_diesel = curated_pkl[(curated_pkl["id_equipo"] == "900") & (curated_pkl["nombre_producto"] == "Formula Diesel")] 
co_477_nafta = curated_pkl[(curated_pkl["id_equipo"] == "477") & (curated_pkl["nombre_producto"] == "VP Nafta")] 
co_248_diesel = curated_pkl[(curated_pkl["id_equipo"] == "248") & (curated_pkl["nombre_producto"] == "D500")] 

In [ ]:
sd_900_nafta = seasonal_decompose(
    co_900_nafta['volumen_escl'].astype(float), 
    freq=60*24
)
sd_900_diesel = seasonal_decompose(
    co_900_diesel['volumen_escl'].astype(float), 
    freq=60*24
)
sd_477_nafta = seasonal_decompose(
    co_477_nafta['volumen_escl'].astype(float), 
    freq=60*24
)
sd_248_diesel = seasonal_decompose(
    co_248_diesel['volumen_escl'].astype(float), 
    freq=60*24
)

In [ ]:
# Rangos de tiempo dummy para marcar turnos, fines de semana en los gráficos
daily_range = pd.date_range(start='2021-02-22', end='2021-04-01', freq='D')
hourly_range = pd.date_range(start='2021-02-22', end='2021-04-01', freq='H')

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False, figsize=(20, 8))
# Semana vs fines de semana
g = seaborn.lineplot(
    data=sd_900_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[0]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range if d.weekday() in [0,5]]
g = seaborn.lineplot(
    data=sd_477_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[1]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range if d.weekday() in [0,5]]

g = seaborn.lineplot(
    data=sd_900_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[2]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range if d.weekday() in [0,5]]
g = seaborn.lineplot(
    data=sd_248_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[3]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range if d.weekday() in [0,5]]
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False, figsize=(20, 8))
# Diario
g = seaborn.lineplot(
    data=sd_900_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[0]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range]
[g.axes.axvline(d, color='darkgreen') for d in daily_range if d.weekday() == 0]
g = seaborn.lineplot(
    data=sd_477_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[1]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range]
[g.axes.axvline(d, color='darkgreen') for d in daily_range if d.weekday() == 0]

g = seaborn.lineplot(
    data=sd_900_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[2]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range]
[g.axes.axvline(d, color='darkgreen') for d in daily_range if d.weekday() == 0]
g = seaborn.lineplot(
    data=sd_248_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[3]
)
[g.axes.axvline(d, color='lightgreen') for d in daily_range]
[g.axes.axvline(d, color='darkgreen') for d in daily_range if d.weekday() == 0]
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False, figsize=(20, 8))
# Horario laboral habitual
g = seaborn.lineplot(
    data=sd_900_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[0]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour in [9, 18]]
g = seaborn.lineplot(
    data=sd_477_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[1]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour in [9, 18]]

g = seaborn.lineplot(
    data=sd_900_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[2]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour in [9, 18]]
g = seaborn.lineplot(
    data=sd_248_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[3]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour in [9, 18]]
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False, figsize=(20, 8))
# Turnos
g = seaborn.lineplot(
    data=sd_900_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[0]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour == 9]
[g.axes.axvline(h, color='lightblue') for h in hourly_range if h.hour == 13]
[g.axes.axvline(h, color='pink') for h in hourly_range if h.hour == 18]
g = seaborn.lineplot(
    data=sd_477_nafta.seasonal[15000:], 
    color='dodgerblue',
    ax=axes[1]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour == 9]
[g.axes.axvline(h, color='lightblue') for h in hourly_range if h.hour == 13]
[g.axes.axvline(h, color='pink') for h in hourly_range if h.hour == 18]

g = seaborn.lineplot(
    data=sd_900_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[2]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour == 9]
[g.axes.axvline(h, color='lightblue') for h in hourly_range if h.hour == 13]
[g.axes.axvline(h, color='pink') for h in hourly_range if h.hour == 18]
g = seaborn.lineplot(
    data=sd_248_diesel.seasonal[15000:], 
    color='limegreen',
    ax=axes[3]
)
[g.axes.axvline(h, color='lightgreen') for h in hourly_range if h.hour == 9]
[g.axes.axvline(h, color='lightblue') for h in hourly_range if h.hour == 13]
[g.axes.axvline(h, color='pink') for h in hourly_range if h.hour == 18]
plt.show()

A grandes rasgos se puede observar:
- Abastecimientos en horarios laborables habituales.
- Mayores consumos en horarios nocturnos.

## Conclusiones

- En principio se llevará a cabo un modelo por centro operativo y producto. En una segunda instancia, se probará un modelo con todos los centros operativos y con hiperparámetros, es por esto que se incluyó la industria en el datadet. Se tendrá en cuenta un período de tiempo en particular para realizar la predicción, para esto será de gran utilidad la descomposición de las series temporales.